1.3.2. Phase 2: Policy Document Indexing for RAG

Index benefits policy PDFs for Retrieval-Augmented Generation (RAG) to enable
efficient policy retrieval.

Text Extraction: Use PyPDF2 to extract text from policy PDFs, removing noise
(e.g., page numbers).

Text Preprocessing: Segment text into chunks (e.g., 200 words per chunk).

Vector Store Creation: Build a Chroma vector store using embeddings from 
sentence-transformers or OpenAI.

Retrieval Testing: Test retrieval with 5 HR policy queries (e.g., “What is the
eligibility for Tuition Reimbursement?”) and evaluate chunk relevance.

In [2]:
!pip install PyPDF2

In [105]:
from PyPDF2 import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter,CharacterTextSplitter
import os
import pandas as pd

directory_path = './assets/benefits/'  # Replace with the actual directory path

pdf_rows = []
text_splitter = CharacterTextSplitter(separator='\n\n',chunk_size=200,chunk_overlap=0)

for filename in os.listdir(directory_path):
    if filename.endswith('.pdf'):
        full_path = os.path.join(directory_path, filename)
        print(f"Found PDF: {full_path}")
        reader = PdfReader(full_path)
        total_pages = len(reader.pages)
        print(f"Total pages: {total_pages}")
        for i in range(total_pages):
            chunk_num = 0
            page_text = reader.pages[i].extract_text()
            page_text = page_text.replace("Introduction", '')
            chunks = text_splitter.split_text(page_text)
            for c in chunks:
                pdf_rows.append({'policy': filename, 'page': i, 'chunk': chunk_num,'text': c})
                chunk_num+=1
policy_pdfs = pd.DataFrame(pdf_rows)

Found PDF: ./assets/benefits/gym-policy.pdf
Total pages: 7
Found PDF: ./assets/benefits/life-insurance-policy.pdf
Total pages: 8
Found PDF: ./assets/benefits/childcare-policy.pdf
Total pages: 8
Found PDF: ./assets/benefits/401k-retirement-policy.pdf
Total pages: 9
Found PDF: ./assets/benefits/work-from-home-policy.pdf
Total pages: 7
Found PDF: ./assets/benefits/vacation-policy.pdf
Total pages: 7
Found PDF: ./assets/benefits/health-insurance-policy.pdf
Total pages: 7
Found PDF: ./assets/benefits/tuition-reimbursement-policy.pdf
Total pages: 8


In [106]:
policy_pdfs['text'][3]

'In addition to traditional gym memberships, all TechLance employees and their families have\ncomplimentary access to our curated online ﬁtness platform, which includes workout videos,\nwellness content, and live virtual classes. We also provide premium subscriptions to popular\nﬁtness apps such as Peloton Digital, Nike Training Club, and Headspace for meditation and\nmindfulness training.\nEnrollment Process and Payment Options\nGetting started with our ﬁtness beneﬁts is straightforward, though it requires some planning and\ndocumentation. All active full-time and part-time employees working at least 20 hours per week\nare eligible for ﬁtness beneﬁts immediately upon hire, with no waiting period or probationary\nrequirements.\nTo enroll, employees ﬁrst browse available ﬁtness partners through our beneﬁts portal,\ncomparing locations, amenities, class schedules, and member ratings. Once you’ve selected a\npreferred facility, you’ll need to complete the enrollment process through our po